ibss마감결과의 오류를 검색 (수수료 정산작업전에 사용)
--- goms와 비교/ 월납오류 검출

In [23]:
import pandas as pd
import GladUtil as gd
pd.set_option('display.max_rows', 1000)
대상 = ['장기','생보','일반','자동차']
미유지 = ['실효','해지','철회','취소']

path_ibss ='d:/글로벌백업(180329)/04.수수료작업/202111수수료/'
ibss_file = []
ibss_file.append('마감실적리스트(장기)-202112230917.xlsx')
ibss_file.append('마감실적리스트(생보)-202112230917.xlsx')
ibss_file.append('마감실적리스트(일반)-202112230917.xlsx')
ibss_file.append('마감실적리스트(자동차)-202112230917.xlsx')
goms_file = 'goms.xlsx'

gb = '장기'
date_cond = '2021-11-01'
 
in_file = ibss_file[대상.index(gb)]
df = pd.read_excel(path_ibss+in_file, header=0, dtype={'증권번호':object})

goms_file = 'goms.xlsx'
df2 = pd.read_excel(path_ibss+goms_file,header=0, dtype={'증권번호':object})

In [24]:
if gb in ('일반','자동차'):
    df['보종'] = gb        
    if gb == '자동차':
        df.rename(columns={'개시일':'계약일','보험료':'월납보험료','납입':'구분'},inplace=True)
        df2['상품종류'] = gb
    else:
        df.rename(columns={'납입보험료':'월납보험료'},inplace=True)
    
''' ibss - 당월건'''
df_ibss = df[['보험회사','증권번호','담당','보종','구분','계약일','월납보험료']].copy()
df_ibss = df_ibss[df_ibss['계약일'] >= date_cond]

''' goms - 해당 보종'''
df_goms = df2[df2['계약종류'] == gb][['보험사','증권번호','수금사원명','계약종류','상품종류','계약상태','계약일자','초회보험료']].copy()

''' join 및 값 비교'''
df_ply = pd.merge(df_ibss, df_goms, how = 'outer',
            on=['증권번호'],indicator = True).reset_index().astype({"_merge": str}).fillna(0)
df_ply.drop(columns='index',inplace=True)

df_ply['사원비교']= ["일치" if str(x)[:2] == str(y)[:2]  else "불일치" 
                 for x,y in zip(df_ply['담당'],df_ply['수금사원명'])]

df_ply['보종비교']= ["일치" if ( str(x) in str(y) or str(y) in str(x) )  else "불일치" 
                 for x,y in zip(df_ply['보종'],df_ply['상품종류'])]

df_ply['증번비교']= ["일치" if x == 'both'  else ( 'ibss만 존재' if x == 'left_only' else "goms만 존재") 
                 for x in df_ply['_merge'] ]

df_ply['회사비교']= ["일치" if str(x)[:2] == str(y)[:2]  else "불일치" 
                 for x,y in zip(df_ply['보험회사'],df_ply['보험사'])]

exp = f"(구분 in {미유지}) and 월납보험료 == 0 "
df_temp = df.query(exp)[['보험회사','증권번호','담당','보종','구분','계약일','월납보험료']]
df_temp['보험료비교'] = '월납오류'

exp  = f" ( 사원비교 != '일치') or (보종비교 != '일치') or (증번비교 != '일치') "
df_error = pd.concat([df_ply.query(exp),df_temp])

df_error.fillna('').to_excel(f'c:/temp/ibss_마감오류_{gb}.xlsx',index = False)